In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip 
from branca.element import Element

In [ ]:
import geopandas as gpd

# Read shapefile (replace with your local path)
gdf = gpd.read_file('STEADy_May2024.shp')

# Quick look at the data
print(gdf.head())


In [ ]:
gdf.columns.values

In [ ]:
equity_solar_gdf = gdf[
    (gdf['IRA_CEJST'] >= 50) &              # Justice40 disadvantaged areas (>50% overlap)
    (gdf['P_Poverty'] >= 20) &              # High poverty (≥20% poverty rate)
    (gdf['NPV_res_m'] > 0) &                # Positive residential solar NPV
    (gdf['IRA_LIC_C1'] >= 50)               # ≥50% overlap with LIC category 1 areas
]

print(f"Total equity emphasis areas: {len(equity_solar_gdf)}")


In [ ]:
#Justice40 (initiative to aims to ensure 40% of overall federal investments flow to disavantaged communities, focuses on climate
# change,clean energy,clean transit,affordable housing,workforce development,legacy pollution,clean water and water infrastructure)

#CEJST is a mapping tool that desginates census tracts as 'disadvantaged'
#IRA_CEJST-Threshold(>=50): Identifying tracts where at least half of the land is classified as disadvantaged by Justice40

#P_Poverty-Percent of population living below the federal poverty line
#P_Poverty(>=20),selecting tracts where at least 20% of residents live in poverty,common marker for economic disadvantage

#NPV_res_m-Residential Solar Net Present Value: avg. long-term financial return for installing residential solar
#NPV_res_m-Threshold(>0) dollar value( positive=good investment)(NREL dGEN model)

#IRA_LIC_C1- Low-Income Category 1 Overlap: percentage of land area in the tract that qualifies as low income under Inflation Reduction Act(IRA)
#bonus credit program(category1)
#IRA_LIC_C1-Threshold(>=50): selecting tracts where at least half of the area qualifies for low income community(10% bonus tax credit eligible)

#ITC(Investment tax credit) criteria:
#1-(50-100%) of tract is eligible for either 'energy community adder' or the 'low income community adder' or 'low income tribal land adder'
#2-(50-100%) of tract is eligible for 'energy community adder' AND the 'low income community adder' or 'low income tribal land adder'

In [ ]:
top_equity_areas = equity_solar_gdf.sort_values('NPV_res_m', ascending=False)

# Display the top 10 tracts
top_equity_areas[['GEOID', 'CountyName', 'NPV_res_m', 'P_Poverty', 'IRA_CEJST']].head(10)


In [ ]:
# Convert relevant columns to numeric and handle 'None' or missing values explicitly
gdf['Median_inc'] = pd.to_numeric(gdf['Median_inc'], errors='coerce')
gdf['NPV_res_m'] = pd.to_numeric(gdf['NPV_res_m'], errors='coerce')

# Drop rows with missing values in these critical columns
gdf_filtered = gdf.dropna(subset=['Median_inc', 'NPV_res_m'])

# Filter to low-income areas (lowest 25%) with positive residential solar NPV
low_income_threshold = gdf_filtered['Median_inc'].quantile(0.25)
low_income_solar_installed = gdf_filtered[
    (gdf_filtered['Median_inc'] < low_income_threshold) & 
    (gdf_filtered['NPV_res_m'] > 0)
]

In [ ]:
# Project to a standard U.S.-friendly CRS (EPSG:5070 = USA_Contiguous_Albers_Equal_Area_Conic)
gdf = gdf.to_crs(epsg=5070)
low_income_solar = low_income_solar_installed.to_crs(epsg=5070)

#US State borders from built-in natural dataset
states=gpd.read_file('ne_110m_admin_1_states_provinces.shp')
states=states[states['admin'] == 'United States of America']
states=states.to_crs(epsg=5070)

# Define bounding box for continental US (approximate)
minx, miny, maxx, maxy = -2500000, 100000, 2500000, 3200000

# Plot
fig, ax = plt.subplots(figsize=(14, 10))
gdf.boundary.plot(ax=ax, linewidth=0.05, color='gray')
states.boundary.plot(ax=ax, linewidth=0.6, color='black')
low_income_solar.plot(ax=ax, color='red', alpha=0.6)

ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)
ax.set_title("Low-Income Areas with Viable Residential Solar Projects (Continental U.S.)", fontsize=15)
ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
#Filter just for Alaska- Tracts that are either low income or have a viable solar impact

ak_solar = gdf[
    (gdf['StateName']== 'Alaska') &
    (

        (gdf['Median_inc'] < low_income_threshold) |
        (gdf['NPV_res_m'] > 0)
    )
]

In [ ]:
fig, ax= plt.subplots(figsize= (10,6))

gdf[gdf['StateName'] == 'Alaska'] \
    .to_crs(epsg=5070) \
    .boundary.plot(ax=ax, linewidth = 0.05, color='gray')


states[states['name'] == 'Alaska'] \
    .to_crs(epsg=5070) \
    .boundary.plot(ax=ax, linewidth = 0.6, color='black')

ak_solar.to_crs(epsg=5070).plot(ax=ax, color='red', alpha= 0.6)
ax.set_title('Low-Income or Solar Viable Tracts- Alaska', fontsize = 14)
ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
hawaii_proj = gdf[gdf['StateName'] == 'Hawaii'].to_crs(epsg=5070)

print(hawaii_proj.total_bounds)

In [ ]:
#Filter just for Hawaii- Tracts that are either low income or have a viable solar impact

hi_solar = gdf[
    (gdf['StateName']== 'Hawaii') &
    (

        (gdf['Median_inc'] < low_income_threshold) |
        (gdf['NPV_res_m'] > 0)
    )
]

In [ ]:
fig, ax= plt.subplots(figsize= (40,15))

gdf[gdf['StateName'] == 'Hawaii'] \
    .to_crs(epsg=5070) \
    .boundary.plot(ax=ax, linewidth = 0.05, color='gray')


states[states['name'] == 'Hawaii'] \
    .to_crs(epsg=5070) \
    .boundary.plot(ax=ax, linewidth = 0.6, color='black')

ax.set_xlim(-7123290,-5976582)
ax.set_ylim(1530055, 3894289)

hi_solar.to_crs(epsg=5070).plot(ax=ax, color='red', alpha= 0.6)
ax.set_title('Low-Income or Solar Viable Tracts- Hawaii', fontsize = 14)
ax.axis('off')
plt.tight_layout()
plt.show()

## Create Maps that are Interactive

In [ ]:
# ensure the columns are numeric

cols= ['Median_inc', 'NPV_res_m', 'P_Poverty','IRA_CEJST','IRA_LIC_C1']
for col in cols:
    gdf[col] = pd.to_numeric(gdf[col], errors='coerce')

# Drop row with missing values in relevant columns

# gdf2= gdf.dropna(subset=cols)


In [ ]:
# gdf.loc[(gdf.StateName == 'Alaska')|(gdf.StateName == 'Hawaii')][['StateName','Median_inc', 'NPV_res_m', 'P_Poverty','IRA_CEJST','IRA_LIC_C1']]

In [ ]:
# filter for the US Map

filtered_US = gdf[
    (gdf['Median_inc'] < low_income_threshold) &
    (gdf['NPV_res_m'] > 0)
].copy()

#filter for Alaska Map
filtered_AK = gdf[
    (gdf['Median_inc'] < low_income_threshold) |
    (gdf['NPV_res_m'] > 0)
].copy()

#filter for Hawaii map
filtered_HI = gdf[
    (gdf['Median_inc'] < low_income_threshold) |
    (gdf['NPV_res_m'] > 0)
].copy()

# Reproject to WGS84 for folium(this ensures that the coordinates in file are expressed as longitude and latitude)

filtered_US = filtered_US.to_crs(epsg=4326)
filtered_AK = filtered_AK.to_crs(epsg=4326)
filtered_HI = filtered_HI.to_crs(epsg=4326)

In [ ]:
# Tooltip Creation

tooltip_fields = ['StateName','CountyName','IRA_LIC_C1','IRA_CEJST','P_Poverty','Median_inc']
tooltip_aliases = ['State:','County:','IRA_LIC_C1 (%):','IRA_CEJST (%):','Poverty Rate (%):','Median Income ($):']

In [ ]:
# Map creation function

def create_map(df, center, zoom):
    fmap = folium.Map(location=center, zoom_start = zoom, tiles= 'CartoDB positron')
    folium.GeoJson(
        df,
        tooltip=GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases, localize=True),
        style_function=lambda feature: {
            'fillColor':'red',
            'color':'black',
            'weight': 0.5,
            'fillOpacity': 0.6,
        },
        name='Filtered Tracts'
    ).add_to(fmap)
    
    
#  Add a custom legend using folium.Html and folium.Marker
    legend_html = """
    <div style="
        position: fixed;
        bottom: 20px; left: 20px;
        width: 300px;
        background-color: white;
        border: 2px solid gray;
        border-radius: 6px;
        padding: 10px;
        font-size: 14px;
        z-index: 9999;
    ">
    <b>Legend</b><br>
    <span style="color:red;">&#9632;</span> Filtered Census Tract<br><br>
    <b>Field Descriptions:</b><br>
    <b>IRA_LIC_C1:</b> % of tract area in a Low-Income Community (IRA Bonus Credit)<br>
    <b>IRA_CEJST:</b> % of tract area designated disadvantaged under Justice40 (CEJST)<br>
    </div>
    """
    
    fmap.get_root().html.add_child(Element(legend_html))
    
    return fmap 
# Display or save
# fmap.save("interactive_us_map_with_legend.html")
# m

In [ ]:
#Create US map
us_map = create_map(filtered_US, center=[39.5, -98.35], zoom=4)
us_map

In [ ]:
#Create Alaska map
ak_map = create_map(filtered_AK[filtered_AK['StateName']== 'Alaska'], center=[64.2, -149.5], zoom=5)
ak_map

In [ ]:
#Create Hawaii map
hi_map = create_map(filtered_HI[filtered_HI['StateName']== 'Hawaii'], center=[20.8, -156.3], zoom=7)
hi_map

In [ ]:
#Save maps
us_map.save('interactive_us_map.html')
ak_map.save('interactive_alaska_map.html')
hi_map.save('interactive_hawaii_map.html')

## RESOURCES


1. https://data.nrel.gov/submissions/238
2. https://docs.nrel.gov/docs/fy24osti/85722.pdf